# Create Edge Passivation
Passivate material edges by detecting and adding passivants to undercoordinated atoms (atoms with fewer neighbors than bulk).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (nanoribbon) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. When prompted, select the coordination threshold based on the analysis.
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete.
1. Scroll down to view results.

## Notes
1. Coordination-based passivation finds atoms with fewer neighbors than expected
2. Coordination threshold below which passivation is applied, must be set depending on the material
3. Passivant atoms are placed along the direction away from existing bonds


## 1. Prepare the Environment
### 1.1. Set up the notebook
Passivation Configuration parameters are described in [Passivation Configuration](https://github.com/Exabyte-io/made/blob/666a941a190a94979b5295f5039f6a4b9158d7dd/src/py/mat3ra/made/tools/build/passivation/configuration.py#L7).
Coordination search radius shown on the image below:

<img src="https://i.imgur.com/zJjjwHP.png" alt="Coordination search radius" width="400"/>

In [ ]:
# Enable interactive selection of coordination threshold
IS_COORDINATION_SELECTION_INTERACTIVE = False

# Material selection
MATERIAL_INDEX = 0

# Passivation parameters
PASSIVANT = "H"  # Chemical element for passivating atom
BOND_LENGTH = 1.09  # Distance from atom to passivant, in Angstroms

# Undercoordinated atoms search algorithm parameters
COORDINATION_THRESHOLD = 2  # Coordination threshold, below which passivation is applied to the atom
COORDINATION_SEARCH_RADIUS = 3.0  # Distance to look for neighbors for coordination, in Angstroms
MAX_BONDS_TO_PASSIVATE = 1  # Maximum number of bonds to passivate

SYMMETRY_TOLERANCE = 0.1  # Tolerance for symmetry analysis of existing bonds

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [1, 1, 1]  # Structure repeat in view

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("")

### 1.3. Load and preview input material


In [ ]:
from utils.jupyterlite import get_materials
from utils.visualize import visualize_materials

materials = get_materials(globals())
nanoribbon = materials[MATERIAL_INDEX]

if SHOW_INTERMEDIATE_STEPS:
    print("Initial nanoribbon structure:")
    visualize_materials(nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
    visualize_materials(nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

## 2. Analyze Coordination Numbers and Select Threshold
### 2.1. Plot RDF to estimate coordination search radius

In [ ]:
from utils.plot import plot_rdf

plot_rdf(nanoribbon, cutoff=10.0)

### 2.2. GEt coordination numbers and set coordination threshold

In [ ]:
from utils.io import select_coordination_threshold
from mat3ra.made.tools.build.passivation import get_unique_coordination_numbers, PassivationConfiguration

config = PassivationConfiguration(
    slab=nanoribbon,
    passivant=PASSIVANT,
    bond_length=BOND_LENGTH
)
coordination_numbers = get_unique_coordination_numbers(config, cutoff=COORDINATION_SEARCH_RADIUS)
print(f"Unique coordination numbers: {coordination_numbers}")

coordination_threshold = COORDINATION_THRESHOLD
if IS_COORDINATION_SELECTION_INTERACTIVE:
    coordination_threshold = await select_coordination_threshold(
        coordination_numbers, COORDINATION_THRESHOLD
    )

print(f"\nSelected coordination threshold: {coordination_threshold}")
print(f"Atoms with coordination < {coordination_threshold} will be passivated")

## 3. Create target material
### 3.1. Set up passivation configuration and builder


In [ ]:
from mat3ra.made.tools.build.passivation.builders import CoordinationBasedPassivationBuilder, \
    CoordinationBasedPassivationBuilderParameters

builder_params = CoordinationBasedPassivationBuilderParameters(
    shadowing_radius=COORDINATION_SEARCH_RADIUS,
    coordination_threshold=coordination_threshold,
    bonds_to_passivate=MAX_BONDS_TO_PASSIVATE,
    symmetry_tolerance=SYMMETRY_TOLERANCE
)

builder = CoordinationBasedPassivationBuilder(build_parameters=builder_params)

### 3.2. Generate passivated structure


In [ ]:
from mat3ra.made.tools.build.passivation import create_passivation

passivated_nanoribbon = create_passivation(config, builder)

print("\nPassivation completed:")
print(f"Passivant used: {PASSIVANT}")
print(f"Bond length: {BOND_LENGTH} Å")
print(f"Original atoms: {len(nanoribbon.basis.elements.ids)}")
print(f"Final atoms: {len(passivated_nanoribbon.basis.elements.ids)}")

## 4. Visualize the result

In [ ]:
print("Final passivated structure:")
visualize_materials(passivated_nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
visualize_materials(passivated_nanoribbon, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

## 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

passivated_nanoribbon.name = f"{nanoribbon.name} passivated"
set_materials(passivated_nanoribbon)